In [1]:
import pandas as pd
from fuzzywuzzy import fuzz, process
import re
import numpy as np
import os

## The panelists by country

Reading in the panel information Netquest gave us.

### may need to be set

In [3]:
paneldate = "200430"

### needs oversight

In [4]:
outdir = "../out/panel_country/"

In [6]:
paneldir = ("C:/Users/schadem/Box/LAPOP Shared/2_Projects/"+
            "2020 IDB Trust/raw/netquest/"+paneldate+"/")
panelfile = [s for s in os.listdir(paneldir) if s.endswith(".csv")][0]
manualfile = "Manual2.xlsx"
print("loading panel data: ", panelfile)


loading panel data:  VANDUS 180376_Abr_20200430.csv


### this can just do its thing

In [7]:
panels = pd.read_csv(paneldir+panelfile,
                     sep=';', 
#                      skiprows=2891450,
#                      nrows=1000000,
                     na_values=[' ','.'], 
                     encoding='latin1',
                     #error_bad_lines=False
                    )

C:\Users\schadem\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,55,66,203,536,537) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
[k for k in panels.columns if "municipio" in k]
#[k for k in panels.columns if "COUNTRY" in k]

['BO_municipio',
 'DO_municipio',
 'GT_municipio',
 'HN_municipio',
 'NI_municipio',
 'PR_municipio',
 'SV_municipio',
 'CO_municipio',
 'int_municipio_delegacion']

Creating a dictionary of individual country dataframes, and cleaning them up--pick out non-empty variables for each country.

In [9]:
panels_dict = {}
for p in panels.COUNTRY.unique():
    panels_dict[p] = panels[panels.COUNTRY==p].dropna(how='all', axis=1)

In [10]:
panels_dict.keys()

dict_keys(['BR', 'MX', 'AR', 'PE', 'CL', 'CO'])

Where does the data go?

In [11]:
for pais, data in panels_dict.items():
    print("working on "+pais)
    print(data.columns)
    if not os.path.isdir(outdir):
        os.mkdir(outdir)
    data.columns = data.columns.str.strip()
    data.to_csv(outdir+pais+"_netquest-panel.csv", encoding='utf8',index=False)

working on BR
Index(['p_codigo', 'p_sexo', 'panelistAge', 'pv_fecha_modificacion',
       'DESKTOP_RESPONDENT', 'DEVICE', 'str_codigo_postal', 'DESK', 'COUNTRY',
       'BRCSOCIAL', 'BR_cp', 'BR_CPF', 'BR_education_level',
       'BR_education_level_full', 'BR_education_level_hhoushold',
       'BR_education_level_hhoushold_full', 'BR_laboral_situation',
       'BR_laboral_situation_extended', 'BR_laboral_situation_hhousehold',
       'BR_numAutos', 'BR_numBaths', 'BR_numComputer', 'BR_numDishwash',
       'BR_numDryer', 'BR_numDVD', 'BR_numFreezer', 'BR_numFridge',
       'BR_numHousemaid', 'BR_numMicro', 'BR_numMoto', 'BR_numWashmachine',
       'BR_pavement', 'family_sector#1', 'family_sector#2', 'family_sector#3',
       'family_sector#4', 'family_sector#5', 'family_sector#6',
       'family_sector#7', 'family_sector#8', 'family_sector#9',
       'family_sector#10', 'family_sector#11', 'family_sector#12',
       'family_sector#13', 'family_sector#14', 'family_sector#15',
       'fa

PermissionError: [Errno 13] Permission denied: '../out/panel_country/CL_netquest-panel.csv'

Next, bring the variables and labels in order.

In [12]:
manual_dict = pd.read_excel(paneldir+manualfile,skiprows=1,sheet_name=None,)
variables = manual_dict['Variables']
variables.Variable = variables.Variable.str.strip()

In [13]:
levels = pd.concat(
    [manual_dict[k] for k in manual_dict.keys() if "Código" in k]
).fillna(method='ffill')
levels.columns = ["Variable","Valor","Etiqueta"]
levels.Variable = levels.Variable.str.strip()
#levels = pd.read_excel(paneldir+"manual_levels.xlsx")
# this expects a the first and second sheet of netquest background export manual
# with first line (title) removed

In [14]:
print(("Variable" in variables.columns) & ("Variable" in levels.columns))

True


In [15]:
variables_dict = {}
levels_dict = {}
countryexp = re.compile("^([A-Z]{2,2})_")

for pais in panels_dict:
    all_countries = variables.Variable.apply(countryexp.findall).apply(lambda l: l[0].lower() if len(l)>0 else "None").unique()
    other_countries = [p.lower() for p in all_countries if not p==pais]
    var_df = variables[
        variables.Variable.isin(panels_dict[pais].columns)
        #remove variables that don't show up in pais
#         [k for k in variables if k in panels_dict[pais].columns]
#         (variables.Variable.str.lower().str.startswith(pais.lower())) #country-specific var
#                    | (variables.Variable.apply(lambda s: sum([s.lower().startswith(k+"_") for k in all_countries]))==0) #not specific for another country
#                    | variables.Variable.isin(["panelistAge","DESKTOP_RESPONDENT","DEVICE","DESK","TARGET",
#                                               "SOCIODEMOGRAPHICS_DATE_V0","SOCIODEMOGRAPHICS_QUEST_V0",
#                                               "int_municipio_delegacion"
#                                              ])
#                    | variables.Variable.str.contains("PNuevo")
                  ]
                       
    lev_df = levels[levels.Variable.isin(var_df.Variable)]
    
    variables_dict[pais] = var_df
    levels_dict[pais] = lev_df
    
    var_df.to_excel(outdir+pais+'_variables.xlsx',index=False)
    lev_df.to_excel(outdir+pais+'_levels.xlsx',index=False)
    

## needs supervision
Check which variables are dropped at this stage

In [16]:
list(filter(lambda x: sum([x in v[1].Variable.unique() for v in variables_dict.items()])==0, variables.Variable))

['BO_departamento',
 'BO_education_level',
 'BO_education_level_hhousehold',
 'BO_laboral_situation',
 'BO_laboral_situation_hhousehold',
 'BO_municipio',
 'BO_provincia',
 'CO_localidad',
 'CRCSOCIAL',
 'CR_canton',
 'CR_cp',
 'CR_distrito',
 'CR_education_level',
 'CR_education_level_hhousehold',
 'CR_laboral_situation',
 'CR_laboral_situation_hhousehold',
 'CR_NSE_cars',
 'CR_NSE_computer',
 'CR_NSE_flights',
 'CR_NSE_internet',
 'CR_NSE_lights',
 'CR_NSE_maid',
 'CR_NSE_rooms',
 'CR_NSE_washingmachine',
 'CR_NSE_year_car',
 'CR_occupation',
 'CR_occupation_hhousehold',
 'CR_provincia',
 'DO_cp',
 'DO_distrito_municipal',
 'DO_education_level',
 'DO_education_level_hhousehold',
 'DO_laboral_situation',
 'DO_laboral_situation_hhousehold',
 'DO_municipio',
 'DO_provincia',
 'ECCSOCIAL',
 'EC_canton',
 'EC_cp',
 'EC_education_level',
 'EC_education_level_hhousehold',
 'EC_laboral_situation',
 'EC_laboral_situation_hhousehold',
 'EC_NSE_autos',
 'EC_NSE_baths',
 'EC_NSE_book',
 'EC_NSE_